In [1]:
# %pip install pyspark

from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import broadcast, substring, col, avg, monotonically_increasing_id, lit
from pyspark.sql.types import StructType, StructField, StringType

In [2]:
sc = SparkContext(appName="Simple App")
spark = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/18 08:40:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_schema=StructType([StructField('Scorecard', StringType(), True), 
                        StructField('Team 1', StringType(), True), 
                        StructField('Team 2', StringType(), True), 
                        StructField('Winner', StringType(), True),
                        StructField('Margin', StringType(), True), 
                        StructField('Ground', StringType(), True),
                        StructField('Match Date', StringType(), True)
                       ])

df = spark.read.csv('../files/cricketMatchesDataset.csv', header=True)

In [4]:
india_df = df.select(col('Team 1'), 
                     col('Team 2'), 
                     col('Winner'), 
                     col('Ground'), 
                     col('Match Date')
                    ).where((df["Team 1"].contains("India")) | (df["Team 2"].contains("India"))
                           ).withColumn('id', monotonically_increasing_id()).cache()
india_df.show()

+-----------+-----------+-----------+------------+---------------+---+
|     Team 1|     Team 2|     Winner|      Ground|     Match Date| id|
+-----------+-----------+-----------+------------+---------------+---+
|    England|      India|    England|       Leeds|   Jul 13, 1974|  0|
|    England|      India|    England|    The Oval|Jul 15-16, 1974|  1|
|    England|      India|    England|      Lord's|    Jun 7, 1975|  2|
|East Africa|      India|      India|       Leeds|   Jun 11, 1975|  3|
|      India|New Zealand|New Zealand|  Manchester|   Jun 14, 1975|  4|
|New Zealand|      India|New Zealand|Christchurch|   Feb 21, 1976|  5|
|New Zealand|      India|New Zealand|    Auckland|   Feb 22, 1976|  6|
|   Pakistan|      India|      India|      Quetta|    Oct 1, 1978|  7|
|   Pakistan|      India|   Pakistan|     Sialkot|   Oct 13, 1978|  8|
|   Pakistan|      India|   Pakistan|     Sahiwal|    Nov 3, 1978|  9|
|      India|West Indies|West Indies|  Birmingham|    Jun 9, 1979| 10|
|     

In [5]:
total_matches = india_df.count()

total_matches_won_home = india_df.where((india_df["Team 1"] == "India") & (india_df["Winner"] == "India")).count() 
total_matches_won_away = india_df.where((india_df["Team 2"] == "India") & (india_df["Winner"] == "India")).count()
total_matches_won = total_matches_won_home + total_matches_won_away
total_matches_won_percentage = total_matches_won/total_matches

total_matches_lost_home = india_df.where((india_df["Team 1"] == "India") & (india_df["Winner"] != "India") & (india_df["Winner"] != "no result")).count()
total_matches_lost_away = india_df.where((india_df["Team 2"] == "India") & (india_df["Winner"] != "India") & (india_df["Winner"] != "no result")).count()
total_matches_lost = total_matches_lost_home + total_matches_lost_away
total_matches_lost_percentage = total_matches_lost/total_matches

total_matches_tie_home = india_df.where((india_df["Team 1"] == "India") & (india_df["Winner"] == "no result")).count()
total_matches_tie_away = india_df.where((india_df["Team 2"] == "India") & (india_df["Winner"] == "no result")).count()
total_matches_tie = total_matches_tie_home + total_matches_tie_away
total_matches_tie_percentage = total_matches_tie/total_matches

print (f"Total Matches Win Percentage: {total_matches_won_percentage:.0%}")
print (f"Total Matches Lost Percentage: {total_matches_lost_percentage:.0%}")
print (f"Total Matches Tie Percentage: {total_matches_tie_percentage:.0%}")

Total Matches Win Percentage: 51%
Total Matches Lost Percentage: 45%
Total Matches Tie Percentage: 4%


In [6]:
total_matches_won_home_percentage = total_matches_won_home/total_matches_won
total_matches_lost_home_percentage = total_matches_lost_home/total_matches_lost
total_matches_tie_home_percentage = total_matches_tie_home/total_matches_tie

print (f"Home Matches Win Percentage: {total_matches_won_home_percentage:.0%}")
print (f"Home Matches Lost Percentage: {total_matches_lost_home_percentage:.0%}")
print (f"Home Matches Tie Percentage: {total_matches_tie_home_percentage:.0%}")

Home Matches Win Percentage: 66%
Home Matches Lost Percentage: 56%
Home Matches Tie Percentage: 42%


In [7]:
total_matches_won_away_percentage = total_matches_won_away/total_matches_won
total_matches_lost_away_percentage = total_matches_lost_away/total_matches_lost
total_matches_tie_away_percentage = total_matches_tie_away/total_matches_tie

print (f"Away Matches Win Percentage: {total_matches_won_away_percentage:.0%}")
print (f"Away Matches Lost Percentage: {total_matches_lost_away_percentage:.0%}")
print (f"Away Matches Tie Percentage: {total_matches_tie_away_percentage:.0%}")

Away Matches Win Percentage: 34%
Away Matches Lost Percentage: 44%
Away Matches Tie Percentage: 57%


In [8]:
indiaHome_df = india_df.where(col('Team 1').contains('India')).groupBy('Team 1','Team 2').count()
indiaAway_df = india_df.where(col('Team 2').contains('India')).groupBy('Team 1','Team 2').count()
indiaAway_df = indiaAway_df.select(col('Team 2').alias('Team 11'), col('Team 1').alias('Team 22'), col('count').alias("count 2"))

cond = [indiaHome_df["Team 1"] == indiaAway_df["Team 11"], indiaHome_df["Team 2"] == indiaAway_df["Team 22"]]
indiaHomeAway_df = indiaHome_df.join(broadcast(indiaAway_df), cond)

topAdversary_df = indiaHomeAway_df.select(
    col('Team 2').alias('Adversary'), 
    (col('count') + col('count 2')).alias('Total Encounters')
    ).orderBy(col('Total Encounters').desc())

topAdversary_df.show()

print(f"First Total Encounters Adversary: {topAdversary_df.collect()[0]}")
print(f"Fifth Total Encounters Adversary Name: {topAdversary_df.select('Adversary').collect()[4]}")
print(f"Count of how many adversaries names starts with the letter S: {topAdversary_df.where(col('Adversary').startswith('S')).count()}")

+------------+----------------+
|   Adversary|Total Encounters|
+------------+----------------+
|   Sri Lanka|             155|
|    Pakistan|             129|
|   Australia|             128|
| West Indies|             121|
| New Zealand|             101|
|     England|              96|
|South Africa|              77|
|    Zimbabwe|              63|
|  Bangladesh|              33|
|       Kenya|              13|
|      U.A.E.|               3|
|     Ireland|               3|
+------------+----------------+

First Total Encounters Adversary: Row(Adversary='Sri Lanka', Total Encounters=155)
Fifth Total Encounters Adversary Name: Row(Adversary='New Zealand')
Count of how many adversaries names starts with the letter S: 2


In [9]:
indiaHomeWin_df = india_df.where((india_df["Team 1"].contains("India")) & (india_df["Winner"] == "India")).groupBy("Team 1","Team 2").count()
indiaAwayWin_df = india_df.where((india_df["Team 2"].contains("India")) & (india_df["Winner"] == "India")).groupBy("Team 1","Team 2").count()
indiaAwayWin_df = indiaAwayWin_df.select(indiaAwayWin_df["Team 2"].alias("Team 11"), indiaAwayWin_df["Team 1"].alias("Team 22"), indiaAwayWin_df["count"].alias("count 2"))

cond = [indiaHomeWin_df["Team 1"] == indiaAwayWin_df["Team 11"], indiaHomeWin_df["Team 2"] == indiaAwayWin_df["Team 22"]]
indiaHomeAwayWin_df = indiaHomeWin_df.join(broadcast(indiaAwayWin_df), cond)
indiaHomeAwayWin_df = indiaHomeAwayWin_df.select(indiaHomeAwayWin_df["Team 1"], indiaHomeAwayWin_df["Team 2"], (indiaHomeAwayWin_df["count"] + indiaHomeAwayWin_df["count 2"]).alias("Total Wins"))
#indiaHomeAwayWin_df.show()

indiaHomeLost_df = india_df.where((india_df["Team 1"].contains("India")) & (india_df["Winner"] != "India")).groupBy("Team 1","Team 2").count()
indiaAwayLost_df = india_df.where((india_df["Team 2"].contains("India")) & (india_df["Winner"] != "India")).groupBy("Team 1","Team 2").count()
indiaAwayLost_df = indiaAwayLost_df.select(indiaAwayLost_df["Team 2"].alias("Team 11"), indiaAwayLost_df["Team 1"].alias("Team 22"), indiaAwayLost_df["count"].alias("count 2"))

cond = [indiaHomeLost_df["Team 1"] == indiaAwayLost_df["Team 11"], indiaHomeLost_df["Team 2"] == indiaAwayLost_df["Team 22"]]
indiaHomeAwayLost_df = indiaHomeLost_df.join(broadcast(indiaAwayLost_df), cond)
indiaHomeAwayLost_df = indiaHomeAwayLost_df.select(indiaHomeAwayLost_df["Team 1"].alias("Team 11"), indiaHomeAwayLost_df["Team 2"].alias("Team 22"), (indiaHomeAwayLost_df["count"] + indiaHomeAwayLost_df["count 2"]).alias("Total Losses"))
#indiaHomeAwayLost_df.show()

cond = [indiaHomeAwayWin_df["Team 1"] == indiaHomeAwayLost_df["Team 11"], indiaHomeAwayWin_df["Team 2"] == indiaHomeAwayLost_df["Team 22"]]
indiaHomeAwayWinLost_df = indiaHomeAwayWin_df.join(broadcast(indiaHomeAwayLost_df), cond, how="leftouter")
indiaHomeAwayWinLost_df = indiaHomeAwayWinLost_df.select(indiaHomeAwayWinLost_df["Team 2"].alias("Adversary"), indiaHomeAwayWinLost_df["Total Wins"], indiaHomeAwayWinLost_df["Total Losses"]).fillna(0)

indiaHomeAwayWinLost_df.show(50)

+------------+----------+------------+
|   Adversary|Total Wins|Total Losses|
+------------+----------+------------+
|      U.A.E.|         3|           0|
|    Zimbabwe|        51|          12|
|       Kenya|        11|           0|
|     England|        52|          44|
|South Africa|        29|          48|
|     Ireland|         3|           0|
| West Indies|        56|          65|
|   Sri Lanka|        88|          67|
|  Bangladesh|        27|           0|
|   Australia|        45|          83|
|    Pakistan|        52|          77|
| New Zealand|        51|          50|
+------------+----------+------------+



In [10]:
indiaMostHomeGround = india_df.where(india_df["Team 1"].contains("India")).groupBy("Ground").count()
indiaMostHomeGround.orderBy(indiaMostHomeGround['count'].desc()).show()         

indiaMostAwayGround = india_df.where(india_df["Team 2"].contains("India")).groupBy("Ground").count()
indiaMostAwayGround.orderBy(indiaMostAwayGround['count'].desc()).show()  

+------------------+-----+
|            Ground|count|
+------------------+-----+
|           Sharjah|   60|
|           Kolkata|   21|
|             Dhaka|   21|
|         Bengaluru|   20|
|           Toronto|   19|
|             Delhi|   19|
|            Mumbai|   18|
|            Nagpur|   17|
|         Ahmedabad|   16|
|           Cuttack|   16|
|            Mohali|   14|
|            Kanpur|   14|
|Hyderabad (Deccan)|   13|
|            Rajkot|   13|
|          Vadodara|   12|
|           Chennai|   12|
|            Indore|   12|
|            Jaipur|   12|
|              Pune|   11|
|           Gwalior|   10|
+------------------+-----+
only showing top 20 rows

+-------------+-----+
|       Ground|count|
+-------------+-----+
|Colombo (RPS)|   35|
|        Dhaka|   21|
|       Sydney|   17|
|       Harare|   16|
|Port of Spain|   16|
|    Melbourne|   15|
|     Dambulla|   13|
|      Sharjah|   12|
|Colombo (SSC)|   12|
|     The Oval|    9|
|      Karachi|    9|
|     Brisbane|   

In [11]:
india_df = india_df.withColumn("Year", substring("Match Date",-4,4))

indiaMatchesWonPerYear_df = india_df.where(col("Winner") == "India").groupBy("Year").count()
indiaMatchesWonPerYear_df = indiaMatchesWonPerYear_df.select(col("Year"), col("count").alias("Wins")) 
indiaMatchesNotWonPerYear_df = india_df.where(col("Winner") != "India").groupBy("Year").count()
indiaMatchesNotWonPerYear_df = indiaMatchesNotWonPerYear_df.select(col("Year"), col("count").alias("Not Wins"))

indiaMatchesPerYear_df = indiaMatchesWonPerYear_df.join(indiaMatchesNotWonPerYear_df, on="Year")
indiaMatchesPerYear_df.withColumn("Win Percentage", (col('Wins')/(col('Wins') + col('Not Wins')))).orderBy("Year").show(40)

indiaMatchesPerYear_df = indiaMatchesPerYear_df.withColumn("Team", lit("India"))
indiaAvgWin = indiaMatchesPerYear_df.groupBy("Team").agg(avg(col("Wins")))
indiaAvgWin.show()

+----+----+--------+-------------------+
|Year|Wins|Not Wins|     Win Percentage|
+----+----+--------+-------------------+
|1975|   1|       2| 0.3333333333333333|
|1978|   1|       2| 0.3333333333333333|
|1980|   3|       2|                0.6|
|1981|   1|       8| 0.1111111111111111|
|1982|   5|       4| 0.5555555555555556|
|1983|   9|      10|0.47368421052631576|
|1984|   2|       9|0.18181818181818182|
|1985|   9|       6|                0.6|
|1986|  12|      15| 0.4444444444444444|
|1987|  12|      10| 0.5454545454545454|
|1988|  12|       8|                0.6|
|1989|   4|      14| 0.2222222222222222|
|1990|   6|       7|0.46153846153846156|
|1991|   8|       6| 0.5714285714285714|
|1992|   6|      15| 0.2857142857142857|
|1993|  11|       7| 0.6111111111111112|
|1994|  16|       9|               0.64|
|1995|   7|       5| 0.5833333333333334|
|1996|  13|      19|            0.40625|
|1997|  10|      29| 0.2564102564102564|
|1998|  24|      16|                0.6|
|1999|  21|     